In [26]:
import threading
import time
from concurrent.futures import ThreadPoolExecutor
from queue import Queue
from bs4 import BeautifulSoup
import requests
import matplotlib.pyplot as plt

In [18]:
def get_page_urls(pages):
    urls = []
    
    for i in range(pages):
        baseurl = 'https://www.mzitu.com/page/{}'.format(pages)
        html = requests_page(baseurl)
        bf = BeautifulSoup(html.text)
        list = bf.find(class_='postlist').find_all('li')
        for item in list:
            url = item.find('span').find('a')['href']
            urls.append(url)
    return urls

In [24]:
def requests_page(url):
    headers = {
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36'}
    html = requests.get(url,headers=headers)
    if html.status_code == 200:
        return html
    else:
        return None

In [27]:
def download(url):
    html = requests_page(url)
    bf = BeautifulSoup(html.text)
    total = bf.find(class_='pagenavi').find_all('a')[-2].find('span').string
    title = bf.find('h2').string
    image_list = []
    for i in range(int(total)):
        html = requests_page(url + '/%s' % (i + 1))
        bf = BeautifulSoup(html.text)
        img_url = bf.find('img')['src']
        image_list.append(img_url)
    return image_list

In [5]:
def moyu_time(name,delay,counter):
    while counter:
        time.sleep(delay)
        print("%s 开始摸鱼 %s" % (name, time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())))
        counter -= 1

In [7]:
class CustomThread(threading.Thread):
    def __init__(self,queue):
        threading.Thread.__init__(self)
        self.__queue = queue
    
    def run(self):
        while True:
            q_method = self.__queue.get()
            q_method()
            self.__queue.task_done()

In [8]:
def moyu():
    print(" 开始摸鱼 %s" % (time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())))

In [12]:
def queue_pool():
    queue = Queue(5)
    for i in range(queue.maxsize):
        t = CustomThread(queue)
        t.setDaemon(True)
        t.start()
        
    for i in range(20):
        queue.put(moyu)
    queue.join()